In [1]:
import pandas as pd
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt



In [2]:
# Charger le fichier CSV
df = pd.read_csv('C:/Users/sylva/OneDrive/Documents/Exo_Data NLP_Nuage_point/comments.csv')



In [3]:
# Charger le modèle de langue française de Spacy
nlp = spacy.load('fr_core_news_sm')



OSError: [E050] Can't find model 'fr_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Supprimer les stopwords en français
stop_words = stopwords.words('french')
def clean_text(text):
    # Enlever les mentions d'utilisateur et les URL
    text = re.sub(r"http\S+|@\S+", "", text)

    # Enlever la ponctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Convertir en minuscules
    text = text.lower()
    # enlever les chiffres
    text = re.sub(r'\d+', '', text)
    # Enlever les stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])

    return text



In [ ]:
df["event"] = df["event"].apply(clean_text)

# Appliquer l'analyse TF-IDF
tfidf = TfidfVectorizer(tokenizer=lambda text: [token.lemma_ for token in nlp(text)],
                        lowercase=True)
X = tfidf.fit_transform(df['event'])
feature_names = tfidf.get_feature_names_out()

# Ajouter une colonne pour les scores TF-IDF de chaque document
for i, doc in enumerate(df['event']):
    indices = X[i].indices
    scores = X[i].data
    for j in range(len(indices)):
        df.loc[i, feature_names[indices[j]]] = scores[j]

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv('tfidf_scores.csv', index=False)